In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog  # For directory selection
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Splash screen class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)  # Remove window decorations (title bar, etc.)
        self.root.attributes("-alpha", 1)  # Initially transparent

        # Get the screen width and height
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Set the window to be centered
        width = 1300
        height = 450
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.root.geometry(f"{width}x{height}+{x}+{y}")

        self.root.configure(bg="black")
        self.root.wm_attributes("-alpha", 1)

        # Create a label for the logo text
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Helvetica", 62, "bold"), bg="black", fg="orange")
        self.label.pack(expand=True)

        # Start the fade in/out process
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        fade_in_time = 1000  # 3 seconds to fade in
        fade_out_time = 2000  # 3 seconds to fade out
        self.fade_in(fade_in_time, lambda: self.fade_out(fade_out_time, self.close_splash))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close_splash(self):
        self.root.destroy()  # Close the splash screen

# Main GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.01")
        
        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.mode = tk.StringVar(value="TTL")
        self.save_path = ""

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)

        # Create GUI for each port
        self.setup_ports()

        # Add fields for experimenter, experiment name, JSON file path, and Spreadsheet ID
        self.create_controls()

        # Add a canvas for the recording indicator
        self.canvas = tk.Canvas(self.mainframe, width=200, height=200)
        self.canvas.grid(column=1, row=6, columnspan=2, padx=5, pady=5)
        self.recording_circle = None
        self.recording_label = None

        # Check for connected devices immediately
        self.check_connected_devices()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 14, "italic"), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }
            self.port_queues[port_name] = queue.Queue()

        for idx in range(len(port_names)):
            self.mainframe.columnconfigure(idx, weight=1)

    def create_controls(self):
        # Experimenter name
        tk.Label(self.mainframe, text="Your Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)
    
        # Experiment name
        tk.Label(self.mainframe, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=2, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, sticky=tk.W, padx=2, pady=2)

        # JSON file path for Google Sheets API
        tk.Label(self.mainframe, text="JSON File Path:", font=("Helvetica", 10, "bold")).grid(column=0, row=3, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.mainframe, textvariable=self.json_path)
        self.json_entry.grid(column=1, row=3, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.mainframe, text="Browse JSON File", font=("Helvetica", 10), command=self.browse_json)
        self.browse_json_button.grid(column=2, row=3, padx=5, pady=10)

        # Spreadsheet ID field
        tk.Label(self.mainframe, text="Google Spreadsheet ID:", font=("Helvetica", 10, "bold")).grid(column=0, row=4, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.mainframe, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=4, sticky=tk.W, padx=2, pady=2)

        # Browse button for selecting data folder
        self.browse_button = tk.Button(self.mainframe, text="Browse Data Folder", font=("Helvetica", 12, "bold"), bg="gold", fg="blue", command=self.browse_folder)
        self.browse_button.grid(column=2, row=4, padx=5, pady=10)

        # Mode toggle button
        self.toggle_mode_button = tk.Button(self.mainframe, text="Toggle Mode", font=("Helvetica", 12, "bold"), bg="grey", fg="white", command=self.toggle_mode)
        self.toggle_mode_button.grid(column=3, row=6, padx=5, pady=10)

        # Start button
        self.start_button = tk.Button(self.mainframe, text="START", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=1, row=5, padx=5, pady=10)

        # Stop button
        self.stop_button = tk.Button(self.mainframe, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=2, row=5, padx=5, pady=10)

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def toggle_mode(self):
        current_mode = self.mode.get()
        if current_mode == "TTL":
            self.mode.set("RTFED")
            self.toggle_mode_button.config(text="Switch to TTL Mode")
        else:
            self.mode.set("TTL")
            self.toggle_mode_button.config(text="Switch to RTFED Mode")

    def get_device_mappings_by_usb_port(self):
        device_mappings = []
        usb_port_mapping = {
            'usb-0:1.1': 'Port 1',
            'usb-0:1.2': 'Port 2',
            'usb-0:1.3': 'Port 3',
            'usb-0:1.4': 'Port 4',
        }
        try:
            for port, port_name in usb_port_mapping.items():
                device_mappings.append({
                    'serial_port': port,
                    'port_identifier': port_name,
                })
        except Exception as e:
            print(f"Error retrieving device mappings: {e}")
        return device_mappings

    def check_connected_devices(self):
        self.device_mappings = self.get_device_mappings_by_usb_port()

        for mapping in self.device_mappings:
            port_identifier = mapping['port_identifier']
            self.port_widgets[port_identifier]['status_label'].config(text="Ready", font=("Helvetica", 15, "bold"), foreground="green")

    def start_experiment(self):
        mode = self.mode.get()
        if mode == "RTFED":
            self.start_rtfed_mode()
        else:
            self.start_ttl_mode()

    def start_rtfed_mode(self):
        print("Starting RTFED mode...")

    def start_ttl_mode(self):
        print("Starting TTL mode...")

    def update_gui(self):
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        pass

# Main execution
if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.after(3000, splash_screen.close_splash)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
